In [1]:
import string
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.inception_v3 import preprocess_input
import numpy as np
import os
from os import listdir
from keras.layers import Input, Dense, LSTM, Dropout, Embedding, add
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import glob
from numpy import array
import matplotlib.pyplot as plt
import random
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Read captions corresponding to each image and store them in list
filename = "flicker8k-dataset/Flickr8k_text/Flickr8k.token.txt"
file = open(filename, 'r')
doc = file.read()

In [3]:
# Read the captions file. Process each line extract 5 captions for each image and append to list.
descriptions = dict()
for line in doc.split('\n'):
    # Splitting the line by tab space
    tokens = line.split('\t')
    print(tokens)
    # Storing image id and descriptions in different variables
    image_id, image_desc = tokens[0], tokens[1:]
    # Removing the extension of image type from the image id
    image_id = image_id.split('.')[0]
    # Storing all the descriptions as one string
    image_desc = ' '.join(image_desc)
    if image_id not in descriptions:
        descriptions[image_id] = list()
    descriptions[image_id].append(image_desc)

['1000268201_693b08cb0e.jpg#0', 'A child in a pink dress is climbing up a set of stairs in an entry way .']
['1000268201_693b08cb0e.jpg#1', 'A girl going into a wooden building .']
['1000268201_693b08cb0e.jpg#2', 'A little girl climbing into a wooden playhouse .']
['1000268201_693b08cb0e.jpg#3', 'A little girl climbing the stairs to her playhouse .']
['1000268201_693b08cb0e.jpg#4', 'A little girl in a pink dress going into a wooden cabin .']
['1001773457_577c3a7d70.jpg#0', 'A black dog and a spotted dog are fighting']
['1001773457_577c3a7d70.jpg#1', 'A black dog and a tri-colored dog playing with each other on the road .']
['1001773457_577c3a7d70.jpg#2', 'A black dog and a white dog with brown spots are staring at each other in the street .']
['1001773457_577c3a7d70.jpg#3', 'Two dogs of different breeds looking at each other on the road .']
['1001773457_577c3a7d70.jpg#4', 'Two dogs on pavement moving toward each other .']
['1002674143_1b742ab4b8.jpg#0', 'A little girl covered in paint 

['1439282131_3814d6ae04.jpg#2', 'Two dogs in the surf holding on to the same stick .']
['1439282131_3814d6ae04.jpg#3', 'Two dogs in the water fighting over a stick .']
['1439282131_3814d6ae04.jpg#4', 'Two dogs swimming with a stick .']
['1440024115_129212c988.jpg#0', 'A man plays Frisbee with his dog .']
['1440024115_129212c988.jpg#1', 'A man with a Frisbee and a dog in the air with a Frisbee in his mouth .']
['1440024115_129212c988.jpg#2', 'A tri-colored dog jumps and catches a pink frisbeen that a man in shorts has thrown .']
['1440024115_129212c988.jpg#3', 'Small dog catching a Frisbee .']
['1440024115_129212c988.jpg#4', 'The man and dog , which is leaping into the air , are playing Frisbee .']
['1443807993_aebfb2784a.jpg#0', 'A child slides down a colorful slide while another child watches']
['1443807993_aebfb2784a.jpg#1', 'A girl going down a slide .']
['1443807993_aebfb2784a.jpg#2', 'A girl in a pink shirt slides down an inflatable fun slide .']
['1443807993_aebfb2784a.jpg#3', 'A

['2064417101_3b9d817f4a.jpg#3', 'Three greyhound dogs race around a dark track .']
['2064417101_3b9d817f4a.jpg#4', 'Three greyhounds reach the finish line .']
['2064780645_8f28a1529f.jpg#0', 'A group of people standing in front of a setting sun .']
['2064780645_8f28a1529f.jpg#1', 'Children in a field at dusk .']
['2064780645_8f28a1529f.jpg#2', 'Several people are standing near trees at dusk .']
['2064780645_8f28a1529f.jpg#3', 'The setting sun outlines a group of four people and the surrounding trees .']
['2064780645_8f28a1529f.jpg#4', 'The sun is setting behind a group of people .']
['2064790732_219e52e19c.jpg#0', 'A boy without a shirt is running in the street while several people stand behind him .']
['2064790732_219e52e19c.jpg#1', 'A little boy dances with a group of people behind him on a city street .']
['2064790732_219e52e19c.jpg#2', 'A small boy dances on the concrete surrounded by adults .']
['2064790732_219e52e19c.jpg#3', 'The little boy wearing no shirt is running by a crowd 

['2247138288_7355861203.jpg#3', 'A group of people with brooms standing in the snow with a dog in the forefront .']
['2247138288_7355861203.jpg#4', 'Group of people standing on a snow covered ground , some are holding brooms , there is a dog with the group .']
['2247192427_5e106f24a9.jpg#0', 'Children play basketball on a court .']
['2247192427_5e106f24a9.jpg#1', 'four young boys play basketball .']
['2247192427_5e106f24a9.jpg#2', 'Several children are playing a basketball game on a basketball court .']
['2247192427_5e106f24a9.jpg#3', 'Three small boys play basketball indoors .']
['2247192427_5e106f24a9.jpg#4', 'Three small boys playing basketball .']
['224766705_b77996527f.jpg#0', 'A caravan of green and red vehicles going up a mountain .']
['224766705_b77996527f.jpg#1', 'A line of green and red dune buggys on a gravel road near rocks and large pine trees .']
['224766705_b77996527f.jpg#2', 'A line of off road vehicles riding along a trail .']
['224766705_b77996527f.jpg#3', 'A long lin

['241345811_46b5f157d4.jpg#1', 'A football coach speaks with a player .']
['241345811_46b5f157d4.jpg#2', 'A football player in red and white is talking to a coach .']
['241345811_46b5f157d4.jpg#3', 'Coach talks with football player , other players and crowd in background .']
['241345811_46b5f157d4.jpg#4', 'There is a football player in a red jersey on the field talking to his coach .']
['241345844_69e1c22464.jpg#0', 'A coach speaking to a football player while everyone watches .']
['241345844_69e1c22464.jpg#1', 'A football player from OU talks to the coach about the game .']
['241345844_69e1c22464.jpg#2', 'A football player is talking to his coach .']
['241345844_69e1c22464.jpg#3', 'The coaching official and football player are talking in front a crowd and other team members .']
['241345844_69e1c22464.jpg#4', 'The coach talks it over with his quarterback during a timeout .']
['241345864_138471c9ea.jpg#0', 'A boy cheerleader has a girl cheerleader on his knee .']
['241345864_138471c9ea.

...

['3183195653_11b66acb34.jpg#0', 'The brightly dressed middle eastern women are conversing in the market .']
['3183195653_11b66acb34.jpg#1', 'The two people have their heads covered with scarves .']
['3183195653_11b66acb34.jpg#2', 'two woman wearing different colored scarfs over their hair outside .']
['3183195653_11b66acb34.jpg#3', 'Two women with head garb looking at each other in a public area .']
['3183195653_11b66acb34.jpg#4', 'Woman with red headwrap at a market with a woman with a brown headwrap .']
['3183519385_311555d5f5.jpg#0', 'A man is about to fall off his surfboard in a wave .']
['3183519385_311555d5f5.jpg#1', 'A man surfs , the sun behind him , his face in shadow .']
['3183519385_311555d5f5.jpg#2', 'A woman on a surfboard in front of a wave of green water']
['3183519385_311555d5f5.jpg#3', 'Someone is surfing a giant wave .']
['3183519385_311555d5f5.jpg#4', 'The boy is surfing .']
['3183777589_460a4f445b.jpg#0', 'A dog sits on a red blanket with a man in a dark hoodie .']


['3259119085_21613b69df.jpg#3', 'A sledder is making a jump over other kids lying in the snow .']
['3259119085_21613b69df.jpg#4', 'One boy jumps over several other boys laying in the snow with his sled , while one boy sits on one end .']
['3259160693_067ec7ebc3.jpg#0', 'A man break dances as a crowd watches .']
['3259160693_067ec7ebc3.jpg#1', 'A man dances on a mat in front of a crowd .']
['3259160693_067ec7ebc3.jpg#2', 'A man dancing on the street in front of a crowd .']
['3259160693_067ec7ebc3.jpg#3', 'A man stands on one hand in the street while a crowd watches .']
['3259160693_067ec7ebc3.jpg#4', 'The guy in the black jacket is standing on his hand on a checkered rectangle .']
['3259222690_69737f2a6e.jpg#0', 'a man in a beige shirt is operating a video camera next to a girl in a black sweater .']
['3259222690_69737f2a6e.jpg#1', 'A man is setting up a camera on a tripod while a woman stands in the background .']
['3259222690_69737f2a6e.jpg#2', 'A man preparing a camera on a tripod']


['3345025842_bc2082a509.jpg#4', 'The girl in the jacket is on the cement with a sucker in her mouth and a bike to the right .']
['3346040664_5b584e6133.jpg#0', 'A boy playing basketball , trying to block a shot .']
['3346040664_5b584e6133.jpg#1', 'One basketball player in a white uniform jumps up to block another player in a blue uniform with the ball .']
['3346040664_5b584e6133.jpg#2', 'The boy in blue has the ball , and the boy in white is airborne .']
['3346040664_5b584e6133.jpg#3', 'The boys play basketball .']
['3346040664_5b584e6133.jpg#4', 'The opposing basketball players are making a play with an audience nearby .']
['3346289227_198fced308.jpg#0', 'A male on Rollerblades skating down a railing next to stairs .']
['3346289227_198fced308.jpg#1', 'A man is rollerblading down a white railing beside a red brick building .']
['3346289227_198fced308.jpg#2', 'A man is skating down a rail above some steps .']
['3346289227_198fced308.jpg#3', 'A man slides a skateboard down a stair ramp .

['3467282545_273a97b628.jpg#3', 'Three men sit together while one looks at a book .']
['3467282545_273a97b628.jpg#4', 'Three middle eastern men are sitting on the floor by a green wall looking at some paperwork .']
['3467510271_0f57e52768.jpg#0', 'A grey helicopter hovering while someone drops in the ocean .']
['3467510271_0f57e52768.jpg#1', 'A man in a wetsuit divind out of a helicopter low over the ocean .']
['3467510271_0f57e52768.jpg#2', 'A man wearing flippers jumps from a helicopter into the water .']
['3467510271_0f57e52768.jpg#3', 'Diver jumping into ocean from helicopter .']
['3467510271_0f57e52768.jpg#4', 'Man jumping out of a helicopter into the ocean .']
['3467843559_a457ce37b6.jpg#0', 'A group of boys sits near the sidewalk and two are smiling while one makes a face .']
['3467843559_a457ce37b6.jpg#1', 'Three boys are posing for a picture , one of them making a silly face .']
['3467843559_a457ce37b6.jpg#2', 'Three boys pose on sidewalk , one boy holding a camera .']
['34678

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
# Cleaning the image captions
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        # Tokenizing the string
        desc = desc.split()
        # Converting the entire string to lower case
        desc = [word.lower() for word in desc]
        # Removing punctuation from each token
        desc = [w.translate(table) for w in desc]
        # Removing 's and article "A"
        desc = [word for word in desc if len(word)>1]
        # Removing words with numbers
        desc = [word for word in desc if word.isalpha()]
        # Storing the caption as a string
        desc_list[i] =  ' '.join(desc)

In [5]:
# Creating an empty set for vocabulary to store unique words
vocabulary = set()
# Counting the size of vocabulary
for key in descriptions.keys():
    [vocabulary.update(d.split()) for d in descriptions[key]]

print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8763
